In [ ]:
%matplotlib inline
from yellowbrick.model_selection import ValidationCurve
import seaborn as sns
from sklearn import preprocessing
from sklearn import impute
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.svm import SVC,LinearSVC
import matplotlib.pyplot as plt
import warnings
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from yellowbrick.classifier import ROCAUC

warnings.filterwarnings("ignore")

#### Load Data

In [ ]:
df = pd.read_csv("HW3_movie_metadata.csv")

In [ ]:
df.head()

,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,James Cameron,723.00,178.00,0.00,855.00,Joel David Moore,"1,000.00","760,505,847.00",Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,...,"3,054.00",English,USA,PG-13,"237,000,000.00","2,009.00",936.00,7.90,1.78,33000
1,Gore Verbinski,302.00,169.00,563.00,"1,000.00",Orlando Bloom,"40,000.00","309,404,152.00",Action|Adventure|Fantasy,Johnny Depp,...,"1,238.00",English,USA,PG-13,"300,000,000.00","2,007.00","5,000.00",7.10,2.35,0
2,Sam Mendes,602.00,148.00,0.00,161.00,Rory Kinnear,"11,000.00","200,074,175.00",Action|Adventure|Thriller,Christoph Waltz,...,994.00,English,UK,PG-13,"245,000,000.00","2,015.00",393.00,6.80,2.35,85000
3,Christopher Nolan,813.00,164.00,"22,000.00","23,000.00",Christian Bale,"27,000.00","448,130,642.00",Action|Thriller,Tom Hardy,...,"2,701.00",English,USA,PG-13,"250,000,000.00","2,012.00","23,000.00",8.50,2.35,164000
4,Doug Walker,NaN,NaN,131.00,NaN,Rob Walker,131.00,NaN,Documentary,Doug Walker,...,NaN,NaN,NaN,NaN,NaN,NaN,12.00,7.10,NaN,0


#### Explore Data

In [ ]:
df.shape
df.info()
numerical_cols = df._get_numeric_data().columns.tolist()
list(set(['a','b'])-set(['a']))
numerical_features=list(set(numerical_cols)-set(['imdb_score']))
numerical_features
numerical_cols=numerical_features.copy()
numerical_cols.append('imdb_score')
numerical_cols
categorical_features=list(set(df.columns.tolist())-set(numerical_cols))
categorical_features

(5043, 27)

In [ ]:
df_nums = df[numerical_cols]
df_nums.head().T
df_nums.describe().T
df_nums.info()

In [ ]:
(df_nums.isnull().sum()*100/len(df_nums)).sort_values(ascending=False)
df_nums.hist(bins=20)
df_nums.skew().sort_values()
df_nums.isna().sum()
df_nums.shape
df_nums['imdb_score'].quantile(q=[0.4,0.5,0.6])

gross                       17.53
budget                       9.76
aspect_ratio                 6.52
title_year                   2.14
director_facebook_likes      2.06
num_critic_for_reviews       0.99
actor_3_facebook_likes       0.46
num_user_for_reviews         0.42
duration                     0.30
actor_2_facebook_likes       0.26
facenumber_in_poster         0.26
actor_1_facebook_likes       0.14
num_voted_users              0.00
cast_total_facebook_likes    0.00
movie_facebook_likes         0.00
imdb_score                   0.00
dtype: float64

#### Preprocess Data

In [ ]:
df_nums['imdb_cut']=df_nums['imdb_score'].apply(lambda x:0 if x<=6.3 else(1 if x>=6.8 else 'middle'))
df_nums['imdb_cut'].value_counts()
df_nums_1=df_nums.drop(df_nums[df_nums['imdb_cut']=='middle'].index)
df_nums_1['imdb_cut'].value_counts()
df_nums_1.head()

In [ ]:
list=df['director_name'].value_counts().head()
list

In [ ]:
x=df_nums_1.drop(['imdb_score','imdb_cut'],axis=1)
x['director']=df['director_name'].apply(lambda x:x if x in list else 'Other')
y=df_nums_1['imdb_cut'].to_numpy()
x['director'].value_counts()
y.shape

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,shuffle=True,stratify=y,random_state=100)
x_train.sort_index().head()
x_train.shape
x_train.head().T

In [ ]:
encoder=preprocessing.OneHotEncoder(handle_unknown='ignore')
x_train1=pd.DataFrame(encoder.fit_transform(x_train[['director']]).toarray(),columns=encoder.categories_[0].tolist())
x_train1=pd.concat([x_train.reset_index(),x_train1],axis=1)
x_train1=x_train1.drop(['index','director'],axis=1)
x_train1.head().T

,0,1,2,3,4
title_year,"2,008.00","2,006.00","1,999.00","2,013.00","2,011.00"
num_voted_users,"11,704.00","46,076.00","200,293.00","148,327.00","48,019.00"
gross,"183,088.00","104,077.00","10,824,921.00","32,154,410.00","39,026,186.00"
num_user_for_reviews,36.00,88.00,621.00,274.00,153.00
duration,115.00,88.00,89.00,106.00,112.00
aspect_ratio,1.85,1.78,1.85,1.85,1.85
num_critic_for_reviews,57.00,117.00,144.00,450.00,128.00
movie_facebook_likes,702.00,"11,000.00","16,000.00","29,000.00","13,000.00"
actor_1_facebook_likes,697.00,"1,000.00",989.00,"17,000.00",969.00
budget,"15,000,000.00","1,000,000.00","10,000,000.00","30,000,000.00","35,000,000.00"


In [ ]:
encoder.categories_[0].tolist()
from sklearn.compose import ColumnTransformer
col_transformer=ColumnTransformer([('onehot',preprocessing.OneHotEncoder(categories='auto',handle_unknown='ignore'),['director'])],remainder='passthrough')
x_train2=pd.DataFrame(col_transformer.fit_transform(x_train),\
    columns=col_transformer.named_transformers_['onehot'].categories_[0].tolist()+x_train.drop('director',axis=1).columns.tolist())
x_train2.head().T
x_train1.describe().apply(lambda x: x.apply(lambda s: format (s,'.1f'))).T
x_train2.describe().T
x_test.head().T

['Clint Eastwood',
 'Martin Scorsese',
 'Other',
 'Ridley Scott',
 'Steven Spielberg',
 'Woody Allen']

In [ ]:
x_test1=pd.DataFrame(encoder.transform(x_test[['director']]).toarray(),columns=encoder.categories_[0].tolist())
x_test1=pd.concat([x_test.reset_index(),x_test1],axis=1)
x_test1=x_test1.drop(['index','director'],axis=1)
x_test1.head().T

,0,1,2,3,4
title_year,"2,006.00","2,003.00","2,009.00","1,993.00","2,000.00"
num_voted_users,775.00,"105,568.00",961.00,"21,940.00","10,220.00"
gross,"638,951.00","34,468,224.00",NaN,"10,769,960.00","5,725.00"
num_user_for_reviews,19.00,533.00,12.00,256.00,110.00
duration,135.00,109.00,96.00,271.00,103.00
aspect_ratio,1.85,1.85,2.35,1.85,2.35
num_critic_for_reviews,15.00,185.00,4.00,22.00,62.00
movie_facebook_likes,100.00,0.00,211.00,0.00,560.00
actor_1_facebook_likes,51.00,"9,000.00",708.00,854.00,127.00
budget,"1,592,000.00","4,500,000.00","3,500,000.00","25,000,000.00","8,400,000.00"


In [ ]:
x_test2=pd.DataFrame(col_transformer.transform(x_test),columns=x_train2.columns)
x_test2.head().T

,0,1,2,3,4
Clint Eastwood,0.00,0.00,0.00,0.00,0.00
Martin Scorsese,0.00,0.00,0.00,0.00,0.00
Other,1.00,1.00,1.00,1.00,1.00
Ridley Scott,0.00,0.00,0.00,0.00,0.00
Steven Spielberg,0.00,0.00,0.00,0.00,0.00
Woody Allen,0.00,0.00,0.00,0.00,0.00
title_year,"2,006.00","2,003.00","2,009.00","1,993.00","2,000.00"
num_voted_users,775.00,"105,568.00",961.00,"21,940.00","10,220.00"
gross,"638,951.00","34,468,224.00",NaN,"10,769,960.00","5,725.00"
num_user_for_reviews,19.00,533.00,12.00,256.00,110.00


##### fill missing values

In [ ]:
imputer=impute.SimpleImputer(missing_values=np.nan,strategy='median')
imputer.fit(x_train1)
x_train1=pd.DataFrame(imputer.transform(x_train1),columns=x_train1.columns)
x_test1=pd.DataFrame(imputer.transform(x_test1),columns=x_train1.columns)

scale the features

In [ ]:
scaler=preprocessing.MinMaxScaler()
scaler.fit(x_train1)
x_train1=pd.DataFrame(scaler.transform(x_train1),columns=x_train1.columns)
x_test1=pd.DataFrame(scaler.transform(x_test1),columns=x_train1.columns)

##### select feature

In [ ]:
n=len(x_train1.columns)
n

21

In [ ]:
x_train1.corr().mask(x_train1.corr().abs()<0.8,'')

,title_year,num_voted_users,gross,num_user_for_reviews,duration,aspect_ratio,num_critic_for_reviews,movie_facebook_likes,actor_1_facebook_likes,budget,...,cast_total_facebook_likes,director_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,Clint Eastwood,Martin Scorsese,Other,Ridley Scott,Steven Spielberg,Woody Allen
title_year,1.00,,,,,,,,,,...,,,,,,,,,,
num_voted_users,,1.00,,0.82,,,,,,,...,,,,,,,,,,
gross,,,1.00,,,,,,,,...,,,,,,,,,,
num_user_for_reviews,,0.82,,1.00,,,,,,,...,,,,,,,,,,
duration,,,,,1.00,,,,,,...,,,,,,,,,,
aspect_ratio,,,,,,1.00,,,,,...,,,,,,,,,,
num_critic_for_reviews,,,,,,,1.00,,,,...,,,,,,,,,,
movie_facebook_likes,,,,,,,,1.00,,,...,,,,,,,,,,
actor_1_facebook_likes,,,,,,,,,1.00,,...,0.96,,,,,,,,,
budget,,,,,,,,,,1.00,...,,,,,,,,,,


In [ ]:
y_train=y_train.astype(int)
y_test=y_test.astype(int)

In [ ]:
from sklearn.feature_selection import mutual_info_classif,SelectKBest
selector = SelectKBest(mutual_info_classif,k=4).fit(x_train1, y_train)
selector.get_support(indices=True)

array([ 1,  4,  7, 12])

In [ ]:
for i in selector.get_support(indices=True).tolist():
    print(x_train.columns[i])

num_voted_users
duration
movie_facebook_likes
director_facebook_likes


In [ ]:
selected_features=x_train1.columns[selector.get_support(indices=True)].tolist()
selected_features

['num_voted_users',
 'duration',
 'movie_facebook_likes',
 'director_facebook_likes']

In [ ]:
train=x_train1[selected_features].copy()
train['num_critic_for_reviews']=y_train
train.corr().round(2).mask(train.corr().abs()<0.8,'')

,num_voted_users,duration,movie_facebook_likes,director_facebook_likes,num_critic_for_reviews
num_voted_users,1.00,,,,
duration,,1.00,,,
movie_facebook_likes,,,1.00,,
director_facebook_likes,,,,1.00,
num_critic_for_reviews,,,,,1.00


#### Model

#### M1: Selected features Model (Not Necessary, but you can do it to compare with other models)

##### train

In [ ]:

clf1=SVC(kernel='linear',C=100000,gamma='auto')


clf1.fit(x_train1[selected_features], y_train)


SVC(C=100000, gamma='auto', kernel='linear')

##### test

In [ ]:

y_trrain_pred =clf1.predict(x_train1[selected_features])
y_test_pred =clf1.predict(x_test1[selected_features])

##### Validation

In [ ]:
print("Classification Report: \n", metrics.classification_report(y_test, y_test_pred))
print("Accuracy on train:  ",clf1.score(x_train1[selected_features],y_train).round(3),'\n')
metrics.confusion_matrix(y_test,y_test_pred).round(3)


Classification Report: 
               precision    recall  f1-score   support

           0       0.63      0.86      0.73       635
           1       0.78      0.51      0.61       640

    accuracy                           0.68      1275
   macro avg       0.71      0.68      0.67      1275
weighted avg       0.71      0.68      0.67      1275

Accuracy on train:   0.67 



array([[543,  92],
       [316, 324]])

In [ ]:
537/(537+287)
537/(537+98)
clf1.dual_coef_.shape
clf1.n_support_
clf1.intercept_
clf1.coef_
len(x_train1.columns)
clf2=SVC(kernel='rbf',C=1000000,gamma='auto')
clf2.fit(x_train1,y_train)
y_test_pred_all=pd.DataFrame(clf2.predict(x_test1))
print("Classification Report: \n", metrics.classification_report(y_test, y_test_pred_all))

0.6516990291262136

#### M3: with gridserchcv

In [ ]:
cv = StratifiedKFold(5)
kernel=['rbf','linear']
c=[100,10000,1000000]
gamma=[1,0.001,'auto']
param_grid = dict(kernel=kernel,C=c,gamma=gamma)
model=SVC()
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv,scoring='accuracy',refit=True,verbose=2)
grid_result=grid.fit(x_train1[selected_features], y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.4s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.3s
[CV] END ......................C=100, gamma=1, kernel=linear; total time=   0.2s
[CV] END ......................C=100, gamma=1, kernel=linear; total time=   0.2s
[CV] END ......................C=100, gamma=1, kernel=linear; total time=   0.2s
[CV] END ......................C=100, gamma=1, kernel=linear; total time=   0.2s
[CV] END ......................C=100, gamma=1, kernel=linear; total time=   0.2s
[CV] END .....................C=100, gamma=0.001, kernel=rbf; total time=   0.3s
[CV] END .....................C=100, gamma=0.001

In [ ]:
y_pred_2=grid.predict(x_test1[selected_features])
metrics.roc_auc_score(y_test,y_pred_2,multi_class='ovr')

0.7125615157480315